In [ ]:
import time
import random
import sklearn
import numpy as np
import pandas as pd
from scipy import stats
from scipy import sparse
from scipy.linalg import eigh
import matplotlib.pyplot as plt
from scipy.sparse import linalg
from scipy.sparse import coo_array
from sklearn.preprocessing import normalize
from scipy.sparse.linalg import eigs, eigsh
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
# Function to compute the sigma matrix for a given distance matrix and neighborhood size 'k'
def sigma_mat(dist, k):
    # Sort each row of the distance matrix
    dist.sort(axis=1)
    # Select the distance to the k-th nearest neighbor
    dist_vec = np.array([dist[:, k-1]])
    # Compute sigma matrix using outer product of distance vector
    sigma_mat = 2 * np.outer(dist_vec, dist_vec)
    return sigma_mat


# Function to calculate diffusion map coordinates
def DM_coordinates(v1, v2, k):
    # Compute pairwise Euclidean distances for view 1
    distsI = euclidean_distances(v1, v1)
    dist_matI = np.copy(distsI)

    # Compute sigma matrix for view 1
    sigma_matI = sigma_mat(dist_matI, k)

    # Construct kernel matrix K for view 1 (Gaussian kernel)
    KI = np.exp(-distsI**2 / sigma_matI)
    # Remove diagonal to eliminate self-connections
    KI = KI - np.diag(np.diag(KI))

    # Degree matrix for v1
    DI = np.diag(np.sum(KI, axis=1))
    D_leftI = np.diag(np.sum(KI, axis=1)**-0.5)
    D_rightI = np.diag(np.sum(KI, axis=1)**0.5)

    # Random walk Laplacian for view 1
    rand_walk_LI = np.matmul(np.linalg.inv(DI), KI)

    # Symmetric normalization
    RWL_primeI = np.matmul(D_rightI, np.matmul(rand_walk_LI, D_leftI))

    # Eigen decomposition for view 1
    eigenValuesI, eigenVectorsI = eigh(RWL_primeI)
    idxI = eigenValuesI.argsort()[::-1]  # Sort in descending order
    d_lamI = np.diag(eigenValuesI[idxI])
    eig_I = D_leftI @ eigenVectorsI[:, idxI]

    # Compute diffusion map coordinates for view 1
    diffusion_coordinatesI = np.matmul(eig_I, d_lamI)
    diffusion_coordinatesI_df = pd.DataFrame(diffusion_coordinatesI)

    # Repeat the process for view 2
    distsII = euclidean_distances(v2, v2)
    dist_matII = np.copy(distsII)
    sigma_matII = sigma_mat(dist_matII, k)

    KII = np.exp(-distsII**2 / sigma_matII)
    KII = KII - np.diag(np.diag(KII))
    DII = np.diag(np.sum(KII, axis=1))
    D_leftII = np.diag(np.sum(KII, axis=1)**-0.5)
    D_rightII = np.diag(np.sum(KII, axis=1)**0.5)
    rand_walk_LII = np.matmul(np.linalg.inv(DII), KII)
    RWL_primeII = np.matmul(D_rightII, np.matmul(rand_walk_LII, D_leftII))

    eigenValuesII, eigenVectorsII = eigh(RWL_primeII)
    idxII = eigenValuesII.argsort()[::-1]
    d_lamII = np.diag(eigenValuesII[idxII])
    eig_II = D_leftII @ eigenVectorsII[:, idxII]

    diffusion_coordinatesII = np.matmul(eig_II, d_lamII)
    diffusion_coordinatesII_df = pd.DataFrame(diffusion_coordinatesII)

    # Return the diffusion map coordinates for view 1 and view 2
    return diffusion_coordinatesI_df, diffusion_coordinatesII_df


# Function to check for conflicts in matching based on the leading eigenvector
def check_conflicts(n, eigenvec):
    # If there's only one eigenvector, use it
    if eigenvec.shape[1] == 1:
        x_star = eigenvec
    else:
        # Use the eigenvector corresponding to the largest eigenvalue
        d = eigenvec.shape[1]
        x_star = eigenvec[:, (d - 1)]

    # Create pairings of all possible rows and columns
    list_x = range(1, n+1)
    list_y = range(1, n+1)

    pairs = []
    for x in list_x:
        for y in list_y:
            pairs.append((x, y))

    # Create a DataFrame to store the pairs and eigenvector values
    df = pd.DataFrame(pairs)
    df["x_star"] = x_star
    df.columns = ["row_idx", "col_idx", "x_star"]
    df_new = df

    first_point = []
    second_point = []

    # Iteratively find the best matches based on the highest x_star value
    for i in range(n):
        idx = np.argmax(df_new.x_star)
        r_idx = df_new.iloc[idx, 0]
        first_point.append(r_idx)
        c_idx = df_new.iloc[idx, 1]
        second_point.append(c_idx)
        # Remove matched rows and columns from consideration
        df_new = df_new[df_new.row_idx != r_idx]
        df_new = df_new[df_new.col_idx != c_idx]

    # Return one-to-one correspondence pairs
    one_to_one = pd.DataFrame(first_point)
    one_to_one["second_point"] = second_point
    one_to_one.columns = ["first_point", "second_point"]
    return one_to_one


# Function to compute Spearman correlation between two sets of points
def spearman_rate(pairs, v1, v2):
    # Extract indices for first set of points
    idx1 = pairs.iloc[:,0]
    idx1 = idx1 - 1
    vec1 = v1.iloc[idx1, 0]

    # Extract indices for second set of points
    idx2 = pairs.iloc[:,1]
    idx2 = idx2 - 1
    vec2 = v2.iloc[idx2, 0]

    # Compute Spearman correlation between the two sets of vectors
    correlation_coefficient, p_value = stats.spearmanr(vec1, vec2)
    return correlation_coefficient


# Function to generate random data for a path graph structure
def data_generator_path_graph(n):
    # Generate two views of a path graph with random x-coordinates
    view_1 = {'x_coord': np.sort(np.random.uniform(0, 100, n)), 'y_coord': np.repeat(1, n), 'z_coord': np.repeat(0, n)}
    view_1 = pd.DataFrame(view_1)
    view_2 = {'x_coord': np.sort(np.random.uniform(0, 100, n)), 'y_coord': np.repeat(-1, n), 'z_coord': np.repeat(0, n)}
    view_2 = pd.DataFrame(view_2)
    return view_1, view_2


# Function to compute the Kronecker product for sparse matrices
def kronecker_product2(A, B, n):
    # Get non-zero elements in matrices A and B
    a_w = np.where(A != 0)
    b_w = np.where(B != 0)
    r_a = list(a_w[0])
    c_a = list(a_w[1])
    r_b = list(b_w[0])
    c_b = list(b_w[1])

    # Compute the row and column indices for the Kronecker product
    R1 = (np.outer(r_a, np.ones((1, len(r_b)))) * n).flatten() + (np.outer(np.ones((len(r_a), 1)), r_b)).flatten()
    C1 = (np.outer(c_a, np.ones((1, len(c_b)))) * n).flatten() + (np.outer(np.ones((len(c_a), 1)), c_b)).flatten()

    # Compute the values of the Kronecker product
    V1 = (np.outer(A[r_a, c_a].flatten(), np.ones((1, len(r_b))))) * (np.outer(np.ones((len(c_a), 1)), B[r_b, c_b].flatten()))

    # Return the Kronecker product as a sparse matrix
    res = coo_array((V1.flatten(), (R1, C1)), shape = (n*n, n*n))
    return res


# Function to compute the M2 matrix for matching
def compute_M2(X, Y, k):
    # Compute pairwise distances for X
    x_dist = euclidean_distances(X, X)
    idx_x = np.argpartition(x_dist, range(k))[:, k:]
    # Threshold distances and symmetrize
    x_dist[np.arange(idx_x.shape[0])[:, None], idx_x] = 2
    x_dist = 2 - x_dist
    x_dist = 0.5 * (x_dist + x_dist.T)

    # Compute pairwise distances for Y
    y_dist = euclidean_distances(Y, Y)
    idx_y = np.argpartition(y_dist, range(k))[:, k:]
    # Threshold distances and symmetrize
    y_dist[np.arange(idx_y.shape[0])[:, None], idx_y] = 2
    y_dist = 2 - y_dist
    y_dist = 0.5 * (y_dist + y_dist.T)

    # Compute the Kronecker product of distance matrices for matching
    n = x_dist.shape[0]
    M2 = kronecker_product2(x_dist, y_dist, n)
    return M2


# Function to compute the Q matrices for matching
def compute_Qs(X, Y, k):
    # Compute pairwise distances for X and create Q matrix
    x_dist = euclidean_distances(X, X)
    idx_x = np.argpartition(x_dist, range(k))[:, k:]
    x_dist[np.arange(idx_x.shape[0])[:, None], idx_x] = 2
    x_dist = 2 - x_dist
    Q_x = 0.5 * (x_dist + x_dist.T)

    # Compute pairwise distances for Y and create Q matrix
    y_dist = euclidean_distances(Y, Y)
    idx_y = np.argpartition(y_dist, range(k))[:, k:]
    y_dist[np.arange(idx_y.shape[0])[:, None], idx_y] = 2
    y_dist = 2 - y_dist
    Q_y = 0.5 * (y_dist + y_dist.T)

    # Return Q matrices for X and Y
    return Q_x, Q_y

In [ ]:
def matching_M2_mixed(n, k, n_col, num_coords, d, data_func, *args):
    """
    Matches two views by computing diffusion maps, normalizing eigenvectors,
    and evaluating matching pairs using Kronecker products and Spearman correlation.

    Parameters:
    -----------
    n : int
        Number of points in each view.

    k : int
        Number of nearest neighbors for diffusion maps.

    n_col : int
        Number of columns to consider from the input data (i.e., dimensions).

    num_coords : int
        Number of diffusion coordinates to extract.

    d : int
        Number of eigenvectors to consider.

    data_func : function
        Function to generate or provide the data for two views.

    *args : additional arguments
        Any additional arguments to pass to `data_func`.

    Returns:
    --------
    opt_pair : DataFrame
        The optimal matching pairs between points in both views.

    view_1, view_2 : DataFrames
        The two views of the data used for matching.

    num_opt_ev : int
        The index of the eigenvector that yielded the best matching score.

    spearman_opt : float
        The Spearman correlation score for the optimal matching eigenvector.

    V_optimal : ndarray
        The eigenvector that maximized the objective function.

    run_time : float
        Total execution time of the function.
    """

    st = time.time()

    view_1, view_2 = data_func(*args)
    view1 = view_1.iloc[:, 0:n_col]
    view2 = view_2.iloc[:, 0:n_col]

    # Compute diffusion coordinates for both views
    diffusion_coordinatesI_df, diffusion_coordinatesII_df = DM_coordinates(view1, view2, k)

    # Extract leading eigenvectors (ignoring the first coordinate)
    leadEV1 = diffusion_coordinatesI_df.iloc[:, 1:num_coords]
    leadEV2 = diffusion_coordinatesII_df.iloc[:, 1:num_coords]

    # Normalize the eigenvectors
    norm_view1 = normalize(leadEV1, axis=1, norm='l2')
    norm_view2 = normalize(leadEV2, axis=1, norm='l2')

    # Compute Q matrices
    Q_1, Q_2 = compute_Qs(norm_view1, norm_view2, k)

    # Compute eigenvalues and eigenvectors of Q matrices
    e_val_Q1, e_vec_Q1 = eigsh(Q_1, k=d)
    e_val_Q2, e_vec_Q2 = eigsh(Q_2, k=d)

    # Compute Kronecker products of eigenvectors
    V = [np.reshape(np.outer(e_vec_Q1[:, i], e_vec_Q2[:, j]), (n ** 2)) for i in range(d) for j in range(d)]

    pairs = []
    spearman_results = []
    result_matrices = []
    objectives = []

    # Loop through each eigenvector
    for idx in range(d**2):
        eVector = V[-(idx + 1)]
        # Compute matching pairs and Spearman correlation
        pair = check_conflicts(n, eVector.reshape(n * n, 1))
        pairs.append(pair)

        spearman_score = spearman_rate(pair, view_1, view_2)
        spearman_results.append(spearman_score)

        # Compute result matrix and objective result
        v = np.zeros((n * n, 1))
        sorted_pairs = pair.sort_values('first_point')
        for row in range(n):
            col_idx = sorted_pairs.iloc[row, 1] - 1
            idx_in_v = (n * row) + col_idx
            v[idx_in_v] = 1
        P_v = v.reshape(n, n)
        result_matrix = Q_2 @ P_v @ Q_1.T @ P_v.T
        result_matrices.append(result_matrix)

        objective_result = np.trace(result_matrix)
        objectives.append(objective_result)

    # Find the maximum objective function value
    obj_arr = np.array([float(obj) for obj in objectives])
    idx_max = np.argmax(obj_arr)
    max_obj = np.max(obj_arr)

    # Optimal correlation
    V_optimal = V[-(idx_max + 1)]
    spearman_opt = spearman_results[idx_max]
    num_opt_ev = idx_max + 1
    opt_pair = pairs[-(idx_max + 1)]

    et = time.time()
    total_time = et - st
    run_time = total_time

    return opt_pair, view_1, view_2, num_opt_ev, spearman_opt, V_optimal, run_time